# S-Feature Analysis
This is an optional notebook where we will go through the steps of creating random walks using the S-path method. These can be used as a feature when learning HAS-embeddings in the HAS_entity_embeddings notebook. We will consider different implementation decisions and look at the results of using this feature for learning embeddings.

*What is this feature?* --> These random walks are intended to detect structural similarity. I.e. entities with similar types of neighbors are similar.

## Pre-requisite steps to run this notebook
1. You need to run the 1_candidate_label_creation notebook before this notebook.
2. gensim is a dependency. You can install it with `pip install --upgrade gensim`, or if you want to use Anaconda, `conda install -c conda-forge gensim`
3. `conda install -c pytorch faiss-cpu`

In [214]:
import os
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import json
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
import math
import faiss

## parameters

**Embedding model parameters**   
*type_to_profile*: Q-node denoting the type of entities you want to create embeddings for (potentially do this for all types in the dataset and get rid of this parameter)  
*num_walks*: Number of random walks to start at each node with the S-feature walk method   
*walk_length*: Length of random walk started at each node  
*representation_size*: Number of latent dimensions to learn from each node  
*window_size*: Window size of skipgram model  
*workers*: Number of parallel processes  

**File/Directory parameters**  
*item_file*: File path for the file that contains entity to entity relationships (e.g. wikibase-item).  
*label_file*: File path for the file that contains wikidata labels.  
*work_dir*: same work_dir that you specified in the label creation notebook. We'll look for files created by that notebook here. Files created by this notebook will also be saved here.  
*store_dir*: Path to folder containing the sqlite3.db file that we will use for our queries. We will reuse an existing file if there is one in this folder. Otherwise we will create a new one.

In [2]:
# Embedding model params
type_to_profile = "Q5"
num_walks = 10
walk_length = 10
representation_size = 64
window_size = 5
workers = 32

# File/Directory params
data_dir = "./data/wikidata_humans"
item_file = "{}/claims.wikibase-item.tsv.gz".format(data_dir)
label_file = "{}/labels.en.tsv.gz".format(data_dir)
work_dir = "./output/wikidata_humans_v3"
store_dir = "./output/wikidata_humans_v3/temp"

### Process parameters and set up variables / file names

In [3]:
# Ensure paths are absolute
item_file = os.path.abspath(item_file)
label_file = os.path.abspath(label_file)
work_dir = os.path.abspath(work_dir)
store_dir = os.path.abspath(store_dir)
    
# Create directories
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
output_dir = "{}/S_walks_analysis".format(work_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(store_dir):
    os.makedirs(store_dir)
    
walks_file = "{}/s_walks.txt".format(output_dir)

# Setting up environment variables 
os.environ["TYPE"] = type_to_profile
os.environ['ITEM_FILE'] = item_file
os.environ['LABEL_FILE'] = label_file
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(store_dir)
os.environ['LABEL_CREATION'] = "{}/label_creation".format(work_dir)
os.environ['OUT'] = output_dir
os.environ['kgtk'] = "kgtk" # Need to do this for kgtk to be recognized as a command when passing it through a subprocess call

### 1. Create simple embeddings for entities based on the types of their neighbors

These embeddings will have $|\tau|$ dimensions where $\tau$ is the set of distinct types amongst entities that share an edge with entities of type $t$. Each dimension of the embeddings will correspond to a type. The embedding for an entity will be created by filling in counts of neighbors of each type and normalizing each dimension.

#### 1.1 Gather list of entities of the type_to_profile along with the types of their neighbors and the counts of neighbors of those types

In [15]:
!kgtk query -i $ITEM_FILE -i $LABEL_CREATION/type_mapping.tsv --graph-cache $STORE \
-o $OUT/${TYPE}_entity_neighbor_types.tsv \
--match '`'"$ITEM_FILE"'`: (e1)-[]->(e2), type: (e1)-[]->(t1:`'"$TYPE"'`), type: (e2)-[]->(t2)' \
--return 'distinct e1 as node1, t2 as label, count(e2) as node2, printf("%s_%s",e1,t2) as id' \
--order-by 'e1, t2'

In [16]:
!head $OUT/${TYPE}_entity_neighbor_types.tsv | column -t -s $'\t'

node1      label      node2  id
Q10000001  Q11879590  1      Q10000001_Q11879590
Q10000001  Q1288568   1      Q10000001_Q1288568
Q10000001  Q1306755   1      Q10000001_Q1306755
Q10000001  Q1323642   1      Q10000001_Q1323642
Q10000001  Q1549591   3      Q10000001_Q1549591
Q10000001  Q1637706   5      Q10000001_Q1637706
Q10000001  Q185145    1      Q10000001_Q185145
Q10000001  Q28640     1      Q10000001_Q28640
Q10000001  Q3024240   1      Q10000001_Q3024240


order neighbor types by frequency of occurrence

In [18]:
!kgtk query -i $OUT/${TYPE}_entity_neighbor_types.tsv -i $LABEL_FILE --graph-cache $STORE \
-o $OUT/${TYPE}_neighbor_types_by_freq.tsv \
--match 'types: (ent)-[l {label:neigh_type}]->(), `'"$LABEL_FILE"'`: (neigh_type)-[:label]->(type_lab)' \
--return 'distinct neigh_type as node1, type_lab as label, count(distinct ent) as node2, neigh_type as id' \
--where 'type_lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'node2 desc'

In [60]:
!head $OUT/${TYPE}_neighbor_types_by_freq.tsv | column -t -s $'\t'

node1      label                                             node2    id
Q55983715  'organisms known by a particular common name'@en  7958973  Q55983715
Q48264     'gender identity'@en                              5959834  Q48264
Q4369513   'sex of humans'@en                                5958411  Q4369513
Q28640     'profession'@en                                   5059332  Q28640
Q12308941  'male given name'@en                              3432358  Q12308941
Q3624078   'sovereign state'@en                              3162245  Q3624078
Q12737077  'occupation'@en                                   3043379  Q12737077
Q6256      'country'@en                                      2941154  Q6256
Q101352    'family name'@en                                  2486879  Q101352


Trim down neighbor types to top 100

In [4]:
!head -101 $OUT/${TYPE}_neighbor_types_by_freq.tsv > $OUT/${TYPE}_neighbor_types_by_freq_trimmed_100.tsv

Filter out less-common neighbor types

In [5]:
%%time
!kgtk ifexists --input-file $OUT/${TYPE}_entity_neighbor_types_trimmed.tsv --filter-file $OUT/${TYPE}_neighbor_types_by_freq_trimmed_100.tsv \
--output-file $OUT/${TYPE}_entity_neighbor_types_trimmed_100.tsv --input-keys label --filter-keys node1

CPU times: user 6.86 s, sys: 1.27 s, total: 8.13 s
Wall time: 5min 51s


In [43]:
%%time
!kgtk query -i $OUT/${TYPE}_entity_neighbor_types.tsv -i $OUT/${TYPE}_neighbor_types_by_freq_trimmed.tsv --graph-cache $STORE \
-o $OUT/${TYPE}_entity_neighbor_types_trimmed_query.tsv \
--match 'entity: (ent)-[l {label:neigh_type}]->(count_neigh_type), freq: (neigh_type)-[]->()' \
--return 'distinct ent as node1, neigh_type as label, count_neigh_type as node2, l as id' \
--order-by 'node1, node2 desc'

CPU times: user 1min 15s, sys: 13.1 s, total: 1min 29s
Wall time: 28min 14s


In [56]:
!cat $OUT/${TYPE}_entity_neighbor_types.tsv | wc -l

85909406


In [55]:
!cat $OUT/${TYPE}_entity_neighbor_types_trimmed.tsv | wc -l

82669563


In [6]:
!cat $OUT/${TYPE}_entity_neighbor_types_trimmed_100.tsv | wc -l

71446433


Create embeddings in Python

In [7]:
%%time
neigh_types_df = pd.read_csv("{}/{}_entity_neighbor_types_trimmed_100.tsv".format(output_dir, type_to_profile), delimiter = '\t').fillna("")
print("loaded entity neighbor type counts")

neigh_types = neigh_types_df.label.unique()
entities = neigh_types_df.node1.unique()
neigh_type_to_idx = {neigh_types[ix] : ix for ix in range(len(neigh_types))}
ent_to_idx = {entities[ix] : ix for ix in range(len(entities))}
embeddings = np.zeros((len(entities),len(neigh_type_to_idx)), dtype=np.float32)

print("filling in embeddings...")
for ent, neigh_type, count in zip(neigh_types_df['node1'], neigh_types_df['label'], neigh_types_df['node2']):
    embeddings[ent_to_idx[ent], neigh_type_to_idx[neigh_type]] = count

# normalize each dimension
# embeddings -= np.nanmin(embeddings,0)
# embeddings /= [m if m != 0 else 1 for m in np.nanmax(embeddings,0)]

loaded entity neighbor type counts
filling in embeddings...
CPU times: user 2min 40s, sys: 19.9 s, total: 3min
Wall time: 3min


In [28]:
%%time
dim = embeddings.shape[-1]
nlist = int(np.sqrt(embeddings.shape[0]))
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, nlist)
assert not index.is_trained
index.train(embeddings)
assert index.is_trained
index.add(embeddings)


CPU times: user 48min 38s, sys: 3min 14s, total: 51min 53s
Wall time: 2min 45s


In [9]:
%time faiss.write_index(index, "{}/{}_IVFFlat_100.index".format(output_dir, type_to_profile))

CPU times: user 0 ns, sys: 2.47 s, total: 2.47 s
Wall time: 2.47 s


In [124]:
!ls -lh $OUT/${TYPE}_IVFFlat_100.index

-rw-r--r-- 1 nmklein div22 3.1G Apr 13 14:34 /data/profiling/kgtk/entity_profiling/output/wikidata_humans_v3/S_walks_analysis/Q5_IVFFlat_100.index


closest neighbors to Putin

In [140]:
%%time
index.nprobe = 20
distances, neighbors = index.search(embeddings[[ent_to_idx["Q7747"]]],21)
neighbors = [entities[n] for n in neighbors[0]]
print(distances)
print(neighbors)

[[  0. 257. 379. 392. 398. 402. 406. 425. 431. 438. 439. 442. 450. 451.
  456. 458. 459. 461. 463. 466. 467.]]
['Q7747', 'Q855', 'Q1394', 'Q36740', 'Q694826', 'Q458702', 'Q23505', 'Q48990', 'Q76', 'Q107441', 'Q567', 'Q454925', 'Q79822', 'Q61064', 'Q6294', 'Q1030228', 'Q83552', 'Q93031', 'Q33391', 'Q135481', 'Q7604']
CPU times: user 634 ms, sys: 40.7 ms, total: 675 ms
Wall time: 27.4 ms


Testing run-time with varied number of search nodes, neighbors to find, nprobe value

In [70]:
%%time
start_nodes = list(range(1))
distances, neighbors = index.search(embeddings[start_nodes],20)

CPU times: user 513 ms, sys: 49.9 ms, total: 563 ms
Wall time: 23.2 ms


In [71]:
%%time
start_nodes = list(range(10))
distances, neighbors = index.search(embeddings[start_nodes],20)

CPU times: user 3.7 s, sys: 446 ms, total: 4.15 s
Wall time: 166 ms


In [72]:
%%time
start_nodes = list(range(100))
distances, neighbors = index.search(embeddings[start_nodes],20)

CPU times: user 8.14 s, sys: 1.74 s, total: 9.88 s
Wall time: 515 ms


In [121]:
%%time
start_nodes = list(range(1000))
distances, neighbors = index.search(embeddings[start_nodes],51)

CPU times: user 40.9 s, sys: 5.51 s, total: 46.4 s
Wall time: 2.19 s


In [143]:
%%time
start_nodes = list(range(10000))
distances, neighbors = index.search(embeddings[start_nodes],21)

CPU times: user 8min 58s, sys: 3.17 s, total: 9min 2s
Wall time: 34.2 s


In [144]:
%%time
for i in range(10):
    distances, neighbors = index.search(embeddings[list(range(i*1000,(i+1)*1000))],21)

CPU times: user 10min 12s, sys: 5.06 s, total: 10min 17s
Wall time: 30.3 s


In [145]:
%%time
for i in range(20):
    distances, neighbors = index.search(embeddings[list(range(i*500,(i+1)*500))],21)

CPU times: user 11min 5s, sys: 7.8 s, total: 11min 13s
Wall time: 30.4 s


In [167]:
%%time
entity_to_neighs = dict()

index.nprobe = 20
k = 21 # number of nearest neighbors to find for each entity (including itself)
batch_size = 1000
num_batches = math.ceil(len(embeddings) / batch_size)
for i in tqdm(range(num_batches)):
    begin = i * batch_size
    end = min((i+1)*batch_size, len(embeddings))
    search_idxs = list(range(begin,end))
    distances, neighbors = index.search(embeddings[search_idxs],k)
    for j in range(len(search_idxs)):
        ent_idx = search_idxs[j]
        ent = entities[ent_idx]
        nbrs = [entities[nbr_idx] for nbr_idx in neighbors[j] if nbr_idx != ent_idx]
        entity_to_neighs[ent] = nbrs

 90%|█████████ | 7167/7959 [3:06:51<20:55,  1.58s/it]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [191]:
with open("{}/{}_entity_to_neighs_dict.json".format(output_dir, type_to_profile), 'w') as f:
    json.dump(entity_to_neighs, f)

### 3. Perform the random walks

In [245]:
def random_walks_to_file(entity_to_neighs, walks_file, walk_length=10, num_walks=10):
    entities = entity_to_neighs.keys()
    print("num entities to perform walks from: {}".format(len(entities)))
    with open(walks_file, "w") as f:
        for ent in tqdm(entities):
            for i in range(num_walks):
                walk = random_walk_from_node(entity_to_neighs, ent, walk_length)
                f.write("{}\n".format(walk))


# Returns a string of space separated Q-nodes as a walk
def random_walk_from_node(entity_to_neighs, start_ent, walk_length):
    walk = start_ent
    cur_ent = start_ent
    cur_length = 1
    while cur_length < walk_length:
        next_ent = random.choice(entity_to_neighs[cur_ent])
        walk = "{} {}".format(walk, next_ent)
        cur_ent = next_ent
        cur_length += 1
    return walk

In [246]:
%%time
random_walks_to_file(entity_to_neighs, walks_file, walk_length, num_walks)

num entities to perform walks from: 7958973



CPU times: user 10min 57s, sys: 22.1 s, total: 11min 19s
Wall time: 11min 16s


### 4. Let's see what embeddings we learn if we only use this feature
Use Skip-Gram model to learn representations for the entities

In [ ]:
%%time
model = Word2Vec(corpus_file=walks_file, size=representation_size, window=window_size, min_count=0, sg=1, hs=1,
                 workers=workers)
model.wv.save("{}/S_embeddings.kv".format(output_dir))

We want similar entities to have more similar embeddings. This feature aims to capture a measure of structural similarity amongst entities of the same type. Therefore we will compare entities within a type to evaluate the embeddings that are learned with this feature.